In [ ]:
# taken from: https://www.sbert.net/examples/unsupervised_learning/SimCSE/README.html

In [1]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers import models, losses
from torch.utils.data import DataLoader
import torch
import pandas as pd
import sqlite3
model_path = '../models/'

In [2]:
con_rev_en_sent = sqlite3.connect('../finetuing_datasets/review_en_train.db')
df_rev_en_sent = pd.read_sql_query("SELECT note FROM review_en_train", con_rev_en_sent)
df_rev_en_sent.head(n=2)

,note
0,good every day drinker for the $
1,died too fast in palate. poor taste of fruits


In [3]:
# best from: https://www.sbert.net/docs/pretrained_models.html#semantic-search
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
word_embedding_model = models.Transformer(model_name, max_seq_length=128)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

train_sentences = df_rev_en_sent.note.to_list()

train_data = [InputExample(texts=[s, s]) for s in train_sentences]

train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True)

train_loss = losses.MultipleNegativesRankingLoss(model)

In [4]:
mps_device = torch.device('mps')
model.to(mps_device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [5]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,
    show_progress_bar=True
)

model.save(model_path + 'simcse_en')

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/24856 [00:00<?, ?it/s]


KeyboardInterrupt

